In [ ]:
# Hackathon

Some utilities

## Import Utils

In [1]:
!pip install keras
!pip install tables
!pip install scikit-learn



    100% |################################| 337kB 3.0MB/s ta 0:00:01
    100% |################################| 256kB 3.9MB/s ta 0:00:011
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/2c/f7/79/13f3a12cd723892437c0cfbde1230ab4d82947ff7b3839a4fc
Successfully built pyyaml
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 4.6MB 323kB/s eta 0:00:01
    100% |################################| 399kB 3.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [53]:
!pip install scikit-image

    100% |################################| 35.5MB 41kB/s  eta 0:00:01
    100% |################################| 1.6MB 996kB/s eta 0:00:01
    100% |################################| 5.7MB 267kB/s eta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/a0/33/4e/7c9228ea77f8090e895d8d2b76f3b5a76997a5b3edeb4e2c6f
Successfully built networkx
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [75]:
import keras
import h5py as h5
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import itertools
import pickle

from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk

PATH_DATA = '/notebooks/data/full.h5'
PATH_80s = '/notebooks/data/eightieth.h5'

PATH_PREDICT_WITHOUT_GT = '/notebooks/data/pred_students/pred_from_half/pred_eighties_from_half_1_without_gt.h5'
PATH_SUBMIT = '/notebooks/ChallengeHacka/pred_eighties_from_half_1_Groupe6.csv'
PATH_EQUI = '/notebooks/data/equi_Groupe6.h5'
PATH_PREDICT_WITH_GT = 'data/pred_teachers/pred_eighties_from_half_1.h5'

In [76]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion): ##La deuxième partie a une taille proportion
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [77]:
def generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
            batch_idxs = sorted(batch_idxs)
            X = f['S2'][batch_idxs, :,:,:]
            Y = f['TOP_LANDCOVER'][batch_idxs, :]
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 23)

In [78]:
##DataSet Full
dsetfull = h5.File(PATH_DATA)['TOP_LANDCOVER']
labelsfull = dsetfull[0:].astype(int)
data4Dfull = h5.File(PATH_DATA)['S2']

In [79]:
##DataSet eigthies
dset80 = h5.File(PATH_80s)['TOP_LANDCOVER']
labels80 = dset80[0:].astype(int)
data4D80 = h5.File(PATH_80s)['S2']

In [80]:
##PATH Predict
data4Dpredict = h5.File(PATH_PREDICT_WITHOUT_GT)['S2']

In [81]:
##Traitement des labels : 23 -> 14
unique, counts = np.unique(labelsfull, return_counts=True)
unique = pd.DataFrame(unique)
counts = pd.DataFrame(counts)
prop = pd.DataFrame(counts/18698240*100)

labelsCountsfull = pd.concat([unique, counts,prop], axis=1, join_axes=[unique.index])
labelsCountsfull.columns = ['label','count','prop']
labelsCountsfull
minProp = 0.02
labelsCountsFiltered = labelsCountsfull[labelsCountsfull['prop']>minProp]
labelsCountsFiltered = labelsCountsFiltered.reset_index()


In [82]:
equilibre = np.zeros((14,3830))

for i in range(len(labelsCountsFiltered)):
    current = labelsCountsFiltered.iloc[i,1]
    print(current)
    equilibre[i][0] = current
    equilibre[i][1:] = np.where(labelsfull == current)[0][:3829]


1
2
3
4
5
10
11
12
14
15
17
18
19
20


In [10]:
Equidxs = np.zeros(14*3829)
for i in range(14):
    for j in range(3829):
        Equidxs[i*3829+j] = equilibre[i,j+1]
        
Equidxs = np.sort(Equidxs)
Equidxs

array([0.0000000e+00, 1.0000000e+00, 2.0000000e+00, ..., 1.6217190e+07,
       1.6756916e+07, 1.6756980e+07])

In [15]:
##DataSet Equilibré premier chargement
dset = h5.File(PATH_DATA)['TOP_LANDCOVER'][Equidxs,:]
labels = dset[0:].astype(int)
data4D = h5.File(PATH_DATA)['S2'][Equidxs,:,:,:]

In [11]:
##DataSet Equilibré depuis fichier
dset = h5.File(PATH_EQUI)['TOP_LANDCOVER']
labels = dset[0:].astype(int)
data4D = h5.File(PATH_EQUI)['S2']

In [ ]:
plt.hist(labelsfull)

In [13]:
np.unique(labels)

array([ 1,  2,  3,  4,  5, 10, 11, 12, 14, 15, 17, 18, 19, 20])

In [103]:
########### SAuvegarde de la matrice entraînée
print("saving model")
f_myfile = open('equilibre.pickle', 'wb')
pickle.dump(equilibre, f_myfile)
f_myfile.close()

# Read from file
f_myfile = open('equilibre.pickle', 'rb')
Q_function = pickle.load(f_myfile)  # variables come out in the order you put them in
f_myfile.close()

saving model


# Instanciation du model

# Fit

## Essai 1 : features moyenne + stdev ; score = 0.5045

In [104]:
def creation_dataset(data4D,labels):
    data = np.zeros((len(data4D),16))
    for i in range(len(data4D)):
        data[i][0]= np.mean(data4D[i][:][:][0])
        data[i][1]= np.std(data4D[i][:][:][0])
        data[i][2]= np.mean(data4D[i][:][:][1])
        data[i][3]= np.std(data4D[i][:][:][1])        
        data[i][4]= np.mean(data4D[i][:][:][2])
        data[i][5]= np.std(data4D[i][:][:][2])
        data[i][6]= np.mean(data4D[i][:][:][3])
        data[i][7]= np.std(data4D[i][:][:][3])
        
        data[i][8]= np.std((data4D[i][:][:][0]+1) / (data4D[i][:][:][1]+1))
        data[i][9]= np.std((data4D[i][:][:][1]+1) / (data4D[i][:][:][2]+1))        
        data[i][10]= np.std((data4D[i][:][:][2]+1) / (data4D[i][:][:][3]+1))
        data[i][11]= np.std((data4D[i][:][:][3]+1) / (data4D[i][:][:][0]+1))
        
        data[i][12]= np.mean(entropy(np.uint8(data4D[i,:,:,0]), disk(4)))
        data[i][13]= np.mean(entropy(np.uint8(data4D[i,:,:,3]), disk(4)))
        data[i][14]= np.std(entropy(np.uint8(data4D[i,:,:,3]), disk(4)))
        
        data[i][15]= labels[i]
        if i%10000 == 0:
            print(i)
    return pd.DataFrame(data)



In [105]:
##Dataset Equilibre
#dataF = creation_dataset(data4D,labels)
print("Equilibre OK")

##Dataset Eigthies
dataF80 = creation_dataset(data4D80,labels80)


########### SAuvegarde de la matrice 80s
print("saving model")
f_myfile = open('dataset80sF1.pickle', 'wb')
pickle.dump(dataF80, f_myfile)
f_myfile.close()
print("80 OK")

##Dataset Predict
datapFredict = creation_dataset(data4Dpredict,range(len(data4Dpredict)))

########### SAuvegarde de la matrice predict
print("saving model")
f_myfile = open('datasetPredF1.pickle', 'wb')
pickle.dump(dataFredict, f_myfile)
f_myfile.close()
print("Predict OK")


Equilibre OK
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
saving model
80 OK
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
saving model


NameError: name 'dataFpredict' is not defined

In [107]:
########### SAuvegarde de la matrice predict
print("saving model")
f_myfile = open('datasetPredF1.pickle', 'wb')
pickle.dump(datapFredict, f_myfile)
f_myfile.close()
print("Predict OK")


saving model
Predict OK


In [17]:
########### SAuvegarde de la matrice equilibre
print("saving model")
f_myfile = open('datasetEquiF1.pickle', 'wb')
pickle.dump(dataF, f_myfile)
f_myfile.close()




saving model


In [99]:
# # Read from file
# # f_myfile = open('datasetEquiF1.pickle', 'rb')
# # data = pickle.load(f_myfile)  # variables come out in the order you put them in
# # f_myfile.close()

# # Read from file
# f_myfile = open('dataset80sF1.pickle', 'rb')
# data80 = pickle.load(f_myfile)  # variables come out in the order you put them in
# f_myfile.close()

# # Read from file
# f_myfile = open('datasetPredF1.pickle', 'rb')
# datapredict = pickle.load(f_myfile)  # variables come out in the order you put them in
# f_myfile.close()

In [116]:
##Dataset test extrait du jeu equilibre
test = pd.DataFrame(dataF.iloc[:,0:8])
testLabel = dataF.iloc[:,8]

test


,0,1,2,3,4,5,6,7
0,493.406250,273.248352,489.812500,271.164703,497.078125,271.457367,499.500000,272.002594
1,500.953125,273.019073,494.734375,272.435272,491.140625,272.250916,500.453125,271.203033
2,490.781250,273.118591,484.218750,271.281128,489.875000,270.830933,494.953125,271.095276
3,488.156250,270.460358,489.281250,271.825226,489.968750,272.051697,487.187500,271.930023
4,497.203125,271.048615,493.937500,272.164001,487.937500,272.722443,487.218750,272.714386
5,496.968750,270.182465,509.078125,269.057404,504.218750,271.193451,489.687500,273.253784
6,494.062500,271.283905,492.640625,270.700745,499.109375,270.166107,494.625000,268.859955
7,493.843750,271.100830,498.656250,271.500122,501.234375,273.005493,496.484375,271.341064
8,492.031250,269.495087,493.703125,269.849121,494.125000,270.812622,487.828125,270.109619
9,489.015625,269.408997,487.890625,269.479950,488.125000,268.951935,489.578125,268.625366


In [131]:
##Dataset train extrait du jeu 80s
train = pd.DataFrame(dataF80.iloc[0:int(len(dataF80)*1),0:15])
trainLabel = dataF80.iloc[0:int(len(dataF80)*1),15]



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1250.578125,698.812927,1243.421875,726.362854,1250.156250,747.154114,1267.625000,734.413879,901.780427,748.509318,864.590928,1609.620452,50720.156286,5097.385361,3403.006601
1,1592.968750,778.150818,1593.250000,787.745117,1539.906250,791.559570,1491.406250,811.838196,789.158866,797.578171,864.068270,1298.775524,51034.555906,5124.227657,3334.203100
2,1632.718750,895.676514,1622.296875,906.153564,1630.109375,885.237976,1708.421875,877.707031,605.942309,909.905732,925.020874,1881.001741,51018.821055,5099.149407,3266.974258
3,1370.281250,691.569458,1312.281250,705.324280,1267.296875,720.594604,1283.312500,721.255188,879.346132,763.521194,523.313209,1220.916361,51026.405319,5147.953844,3323.210162
4,1375.859375,656.358826,1372.390625,650.016846,1335.875000,654.548096,1288.640625,669.399902,392.826870,666.214377,815.373734,991.289467,51096.457576,5112.972665,3441.918692
5,1226.500000,471.456024,1281.546875,557.399902,1334.843750,645.688477,1388.156250,693.514221,515.104458,596.942157,787.802488,1744.123399,51265.354710,5099.956795,3668.928131
6,1425.718750,989.549438,1383.156250,927.540771,1366.171875,912.155457,1340.250000,865.968262,963.996127,679.339245,984.302461,1028.814241,50418.949880,5106.245261,3348.344357
7,1391.203125,857.283691,1420.328125,896.652954,1416.375000,899.682617,1388.703125,906.276978,734.286383,537.082143,500.402115,1045.881361,51153.379585,5079.885968,3353.725175
8,1381.312500,699.362793,1384.828125,710.943054,1395.859375,701.093628,1376.812500,673.707031,850.203037,550.749004,695.691109,1151.585057,50425.972512,5005.361257,3404.558150
9,1166.500000,568.878967,1170.593750,559.323914,1178.062500,547.621155,1172.687500,534.595947,451.255590,346.600562,396.017358,588.948727,50348.377002,5113.198460,3645.295787


In [156]:
##Dataset train extrait du jeu 80s
dataF80Split = dataF80[dataF80[15] != 6 ]
dataF80Split = dataF80Split[dataF80Split[15] != 4 ]
dataF80Split = dataF80Split[dataF80Split[15] != 9 ]
dataF80Split = dataF80Split[dataF80Split[15] != 11 ]
dataF80Split = dataF80Split[dataF80Split[15] != 13 ]
dataF80Split = dataF80Split[dataF80Split[15] != 14 ]
dataF80Split = dataF80Split[dataF80Split[15] != 15 ]
dataF80Split = dataF80Split[dataF80Split[15] != 17 ]
dataF80Split = dataF80Split[dataF80Split[15] != 18 ]
dataF80Split = dataF80Split[dataF80Split[15] != 20 ]
dataF80Split = dataF80Split[dataF80Split[15] != 21 ]
train = pd.DataFrame(dataF80Split.iloc[0:int(len(dataF80Split)*1),0:15])
trainLabel = dataF80Split.iloc[0:int(len(dataF80Split)*1),15]
np.unique(dataF80Split[15])


array([ 1.,  2.,  3.,  5., 10., 12., 19.])

In [124]:
##Dataset test extrait du jeu prediction
prediction = pd.DataFrame(datapFredict.iloc[:,0:15])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1347.437500,365.043579,1339.203125,369.483307,1344.062500,382.483032,1344.250000,382.905792,324.267335,337.284394,338.156410,479.666144,51013.247318,5008.195542,3662.077322
1,1305.796875,379.181366,1296.906250,371.544128,1305.078125,385.281281,1293.671875,386.236725,272.137728,306.243915,284.765158,434.051976,50116.126218,5054.307720,3546.392016
2,1277.375000,418.117249,1273.703125,406.332733,1289.562500,421.727509,1290.609375,421.043915,276.984088,306.007136,198.308844,662.797242,47672.235426,5059.148835,3158.814046
3,1326.781250,382.865356,1306.812500,379.129669,1279.406250,390.198029,1259.250000,387.099823,221.121889,294.630527,246.418603,396.529511,49134.939647,5075.714516,3362.196947
4,1305.671875,393.949402,1295.359375,396.607849,1277.046875,383.786224,1248.796875,367.041687,306.949392,211.149976,232.497063,363.517776,49248.809753,5026.673817,3560.290692
5,1352.328125,440.455444,1366.421875,435.561066,1360.734375,419.852295,1323.468750,406.279449,417.247713,235.665813,249.314271,505.754985,49540.659730,5076.418517,3331.391497
6,1384.015625,636.355713,1375.437500,583.978577,1368.625000,549.174377,1363.187500,534.125671,420.345888,392.146297,449.481532,890.152827,49796.291084,5074.792760,3621.494944
7,1383.671875,777.343018,1339.390625,723.756775,1330.031250,708.695679,1345.203125,720.121216,756.395832,1166.792512,1034.843326,2821.988165,51010.065985,5149.843218,3605.508159
8,1537.843750,801.029053,1560.515625,785.548218,1622.875000,765.595520,1676.093750,780.799438,476.559997,538.506955,741.440728,1075.733379,51167.017943,5134.535844,3714.673962
9,1588.046875,725.621094,1591.093750,696.973450,1624.125000,763.412354,1669.312500,794.971313,309.358779,416.009985,401.048698,489.536449,50513.860813,5137.857947,3714.823872


In [178]:
dt_tree = RandomForestClassifier(n_estimators=100,max_depth=9,criterion="entropy")
dt_tree.fit(train,trainLabel)
print(dt_tree.feature_importances_)

[0.14484163 0.05638858 0.17409815 0.06519206 0.13462234 0.05928667
 0.13385222 0.07249901 0.02796422 0.028878   0.03124106 0.02351243
 0.03191221 0.0126442  0.00306722]


In [134]:
dt_tree.score(test,testLabel)

ValueError: Number of features of the model must match the input. Model n_features is 15 and input n_features is 8 

In [179]:
labelspredicted = dt_tree.predict(prediction)
labelspredicted = labelspredicted.astype(int)

for i in range(len(labelspredicted)):
    if labelspredicted[i] == 2:
        labelspredicted[i] = 1
        

In [180]:
np.unique(labelspredicted,return_counts=True)

(array([ 1,  3,  5, 12]), array([24130, 47545, 79653, 90372]))

## Prediction routines

In order to submit a result here are some gits

In [89]:
import os 
def prediction_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        X = f['S2'][batch_idxs, :,:,:]
        yield np.array(X)

def build_h5_pred_file(dset, data4D, h5_output_path):
    if os.path.exists(h5_output_path):
        os.remove(h5_output_path)
    f = h5.File(h5_output_path, 'w')
    top_landcover_submit = f.create_dataset("TOP_LANDCOVER", (len(dset), 1), maxshape=(None, 1))
    top_landcover_submit[:, 0] = dset[:,0]
    s2_submit = f.create_dataset("S2", (len(data4D), 16,16,4), maxshape=(None, 16,16,4))
    s2_submit[:,:,:,:] = data4D
    f.close()
    
    return 1

build_h5_pred_file(dset,data4D,PATH_EQUI)

1

In [177]:
#pred_idx = get_idxs(PATH_PREDICT_WITHOUT_GT)
#print(len(pred_idx))


panda_prediction = pd.DataFrame(labelspredicted)
panda_prediction.columns = ['TOP_LANDCOVER']
panda_prediction.head()
panda_prediction.describe()
panda_prediction.to_csv(PATH_SUBMIT, index_label = 'ID')

## Some ideas for monitoring

In [ ]:
def gt_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        Y = f['TOP_LANDCOVER'][batch_idxs, :]
        yield keras.utils.np_utils.to_categorical(np.array(Y), 23)

gt_gen = gt_generator(PATH_PREDICT_WITH_GT, BATCH_SIZE, pred_idx)
gt = []
for elem in gt_gen:
    gt.append(elem)
gt = np.vstack(gt)

In [41]:

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",fontsize=7)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def clean_confusion_matrix(confusion_matrix, classes):
    real_classes = []
    for c in range(len(classes)):
        if np.sum(confusion_matrix[:,c])+np.sum(confusion_matrix[c, :]) != 0:
            real_classes.append(c)
    real_confusion_matrix = np.empty((len(real_classes), len(real_classes)))  
    for c_index in range(len(real_classes)):
        real_confusion_matrix[c_index,:] = confusion_matrix[real_classes[c_index], real_classes]
    return real_confusion_matrix, real_classes

In [ ]:
%matplotlib notebook
from sklearn.metrics import confusion_matrix
y_true = np.argmax(gt, axis=1)
y_pred = np.argmax(prediction, axis = 1)

real_cnf_matrix, real_classes = clean_confusion_matrix(confusion_matrix(y_true, y_pred, labels= range(23)), range(23))
plot_confusion_matrix(real_cnf_matrix, classes = real_classes, normalize=True)